# The Game 
## without interactions with the map or the google sheet

initial conditions with (until now) random numbers:

In [1]:
import numpy as np
import math

In [2]:
# create dummy matrix
n_plots = 4
rows = cols = 80

plot_length = int(rows/n_plots)
dummy_plot = np.ones(plot_length**2).reshape((plot_length, plot_length))
A1 = dummy_plot
A2 = dummy_plot * 2
B1 = dummy_plot * 3
B2 = dummy_plot * 4
large_matrix = np.block([[A1, A2], [B1, B2]])
large_dummy_matrix = np.ones_like(large_matrix)
n = plot_length
coef_matrix = np.array([[1, 2], [3, 4]])
result = np.multiply(large_dummy_matrix, np.kron(coef_matrix, np.ones((n,n))))
matrix_indizes = np.indices((n_plots, n_plots), dtype="uint8") + 1
row_indizes, column_indizes = matrix_indizes[0], matrix_indizes[1]
plot_definition_matrix = np.char.add(row_indizes.astype(np.str), column_indizes.astype(np.str)).astype(np.uint8)
dummy_playing_field_matrix = np.ones(shape=(rows, cols), dtype=np.uint8)
large_plot_definition_matrix = np.multiply(
    dummy_playing_field_matrix,
    np.kron(plot_definition_matrix, np.ones(shape=(plot_length, plot_length)))
)

In [3]:
# what I used from your matrix-dummies-notebook:
lulc_matrix = dummy_playing_field_matrix             # 80 x 80
cooperation_matrix = dummy_playing_field_matrix      # 80 x 80
plot_definition_matrix                               # 4 x 4
tourism_matrix = plot_definition_matrix              # 4 x 4
n_blocks = 4
n_pixel = 20

# other assumtions:
rounds = 10   # number of rounds
teams = 3    # in round 3 - first 3 rounds to grasp whats happening
brexit = 6    # in round 6 - 
increased_timber_prices = 1.1 # +10%

I thought with these dictionaries the change from dummy matrix to the real data should be easy. 

In [4]:
dummy_playing_field_matrix.shape

(80, 80)

In [6]:
# include new dictionary - the ownership dictionary was never used because i assigned each player a block. - this is also necessary for teamwork (at least with this code)
#ownership = {
#    'Forester1': [11, 12, 21, 22],  # Forester 1 owns Plot 11, 12, 21, 22  
#    'Farmer1': [13, 14,23, 24],     # Farmer 1 owns Plot 13, 14, 23, 24 
#    'Farmer2': [31, 32, 41, 42],    # Farmer 2 owns Plot 31, 32, 41, 42 
#    'Forester2': [33, 34, 43, 44]  # Forester 2 owns Plot 33, 34, 43, 44 
#}

landuse = {
    'cattle': 1,
    'sheep': 2,
    'n_forest': 3,
    'c_forest': 4
}

# i modified that because it doesn't have cattle cet
# describes the simplified LULC types 
simplified_lulc_mapping = {

    "Sheep Farming": 1,
    "Native Forest": 2,
    "Commercial Forest": 3,
    "Cattle Farming": 4

}

#number_of_players = len(ownership) + 1 # +1 because of the tourism 
# for more than 5 players there would be a new way to allocate ownership of plots.

A function for the decision if teamwork takes place.  It assumes 4 players allocated as above and that all have to agree to it. 

In [7]:
# assumes four players only / if the four corner player say yes then it's true.
def teamwork(cooperation_matrix):
    teamwork = False
    row, col = cooperation_matrix.shape
    if cooperation_matrix[0][0] == cooperation_matrix[0][col-1] == cooperation_matrix[row-1][0] == cooperation_matrix[row-1][col-1] == True:
        teamwork = True
    return teamwork

In [8]:
team_work = teamwork(cooperation_matrix)
team_work

True

A function returning the number of landuse-pixel for a given matrix (according to the dictionary landuse above). 

In [9]:
# get the total yield for the current map
def yield_map(field):
    tot_cattle = np.count_nonzero(field == simplified_lulc_mapping['Cattle Farming'])
    tot_sheep = np.count_nonzero(field == simplified_lulc_mapping['Sheep Farming'])
    tot_n_forest = np.count_nonzero(field == simplified_lulc_mapping['Commercial Forest'])
    tot_c_forest = np.count_nonzero(field == simplified_lulc_mapping['Cattle Farming'])
    return tot_cattle, tot_sheep, tot_n_forest, tot_c_forest

In [10]:
# get the yield for the current playing field
tot_cattle, tot_sheep, tot_n_forest, tot_c_forest = yield_map(lulc_matrix)

In [11]:
# get the yield for a plot
plot_cattle, plot_sheep, plot_n_forest, plot_c_forest = yield_map(plot_definition_matrix)

In [12]:
# cropping a block out of the map
indices_block_1 = list(range(0, int(rows/math.sqrt(n_blocks))))
indices_block_2 = list(range(int(rows/math.sqrt(n_blocks)), rows))
lulc_matrix[np.ix_(indices_block_1,indices_block_2)].shape

(40, 40)

In [19]:
# global variables for the profit_pp function:
income_farmland_sheep = 30
income_farmland_cattle = 100
income_forest_commercial = 200
income_forest_native = 50
gdp_pc_scotland = 29.600
unempl_rate_scotland = 0.05

In [22]:
row = col = 80
estimate_farmland = 1/2 # half of it is farmland - maybe a bit lower than reality to increase pressure for the game?
estimate_forest = 1/3
number_of_farmer = row*col*income_farmland_sheep/gdp_pc_scotland*estimate_farmland/(1-unempl_rate_scotland)
number_of_forester = row*col*income_forest_commercial/gdp_pc_scotland*estimate_forest/(1-unempl_rate_scotland)
unempl_rate_scotland = 0.05


In [23]:
# uses the number_of_farmers or the numbers_of_forester above and the total amount of money from both farmer or forester as an argument
def unemployment(money, gdp_pc_scottland, number_of_workers):
    unempl_rate_scotland = int(money / gdp_pc_scottland)/number_of_potential_workers
    return unempl_rate_scotland

The idea for the function profit_pp is to try to adapt the prices and save the adapted prices in a list. these equations are based on the starting value with the assumption that they dictate the demand.
The variable brexit says if brexit already happened this round. This would lead to increased_timber_prices. I thought that might slightly reduce all those different variables, becuase I felt it is getting a bit over-complicated. 

The tot_\\<product\> is a list of the total area of the product gained by the function yield_map. 
\\<product\>_pp is a list with the former prices per pixel. 

The multiplying factors are a bit random but also based on the assumpton that the price difference of the production is an indicator on the productivity. i.e. 3 = income_farmland_cattle/income_farmland_sheep. 
Because I didn't want to risk dividing by zero I added a 1. 

In [70]:
# adapt the prices
def profit_pp(round, brexit, increased_timber_prices, tot_sheep, tot_cattle, tot_n_forest, tot_c_forest, cattle_pp, sheep_pp, n_forest_pp, c_forest_pp):      
    #doesn't take tourism effects into account yet. and the equations are pretty random.
    cattle_pp_new = tot_sheep[0] / (1 + tot_cattle[round]*3 + tot_sheep[round])*cattle_pp[0] # a certain demand - + 1 so that its never going to infinity should all land become forest
    sheep_pp_new = sheep_pp[0] # assume sheep can go everywhere, eat everything and no degradation and its profit only influences cattle by competition
    c_forest_pp_new = (tot_c_forest[0] + tot_n_forest[0])/(1 + tot_c_forest_tot[round] * 4 + area_n_forest_tot[round]) * c_forest_pp[0]
    n_forest_pp_new = n_forest_pp[0] # assumes native forest can grow everywhere and its profit only influences the commercial forest through competition in the timber market
    if brexit > round:
        n_forest_new *= increased_timber_prices # less import of wood. 
        c_forest_new *= increased_timber_prices
    return cattle_pp_new, sheep_pp_new, n_forest_pp_new, c_forest_pp_new


In [71]:
from pdb import set_trace

Here I try to calculate the money a farmer earns each **round**. The **tourism** is a factor t calculated with the tourism_matrix
i.e. cattle, sheep, n_forest, c_forest = yield_map(tourism_matrix)
        t = (sheep*2 + cattle*1.2 + n_forest*2)/n_pixel/n_pixel/(1+c_forest*10) The numbers in this equation are just based on discussions on what is how bad. The one is again added so that there is no division through zero. The division by the number of pixel is to norm t a bit. It is then a factor that can be max twice as high. 

**brexit** is again an integer ot the round at which brexit happens. I discussed with the design team that teams could be about 3 and brexit then a few rounds after - i.e. 5. 
        
**teams** is an integer of the round after which teams are allowed. **teamwork** is the result of the function teamwork. 
the area_\\<product> is a list of the area of the individual farmer. It can be calculated by yield map by adressing an indicidual block. 
ie.yield_map(lulc_matrix[np.ix_([*list of pixels 0 to 39*],[*list of pixel 0 to 39*])])
\\<product>_pp is a list of the prices calculated with profit_pp
    
the other arguments are the costs of landuse change. There are no approximations on marcos excel file but we agreed that it should cost something. I thought I'd ask marco next time about it. 

I made up this list it say:
    
     cf_to_nf = 0.5 
    
    nf_to_cf = 0.5
    
    s_to_c = 0.5
    
    s_to_nf = c_to_nf = 1 # must stay the same (brexit calculations for farmer)
    
    c_to_s = 0.5
    
    nf_to_s = -0.1   # assuming farmers can convert native forest to farmland but not commercial forest (sell wood)
    
    nf_to_c = 0.8 
    
These factors define on how profitable the land is on the first year with the new use. i.e. 0.5 means that it only has half the use. 
    
For the mpney_pp_forester function it's the same.
    
subsidies is something that I thought could try to include the tax break for the native forests and the subsidies for the sheep farming. during brexit. That was before the taxation list. I think it could again simplify things. I ust set it to I think it could be something like 0.8.
The tax-breaks might also not concern "normal taxes" but rather in regards to inheri

In [84]:
def money_pp_farmer(round, tourism, teams, brexit, teamwork, area_sheep, area_cattle, area_c_forest, area_n_forest, sheep_pp, cattle_pp, n_forest_pp, c_forest_pp, nf_to_s, nf_to_c, s_to_c, s_to_nf, c_to_s, c_to_nf, subsidies, starting_capital):
    if round == 0: 
        money = starting_capital
    else:   
         # costs of landscape change
        try:
            d_sheep = area_sheep[round] - area_sheep[round-1]
        except: set_trace()
        d_cattle = area_cattle[round] - area_cattle[round-1]
        d_n_forest = area_n_forest[round] - area_n_forest[round-1] # necessary to potentially allow two changes (i.e. a rise or native forests and cattle on cost of sheep )
        m_change = 0
        m_brexit = 0
        if d_n_forest < 0: 
            m_change += min([d_cattle, d_n_forest], key= abs)*nf_to_c*cattle_pp[round]
            m_change += min([d_sheep, d_n_forest], key= abs)*nf_to_s*sheep_pp[round]
        if d_sheep < 0:
            m_change += min([d_cattle, d_sheep], key= abs)*s_to_c*cattle_pp[round]
            m_change += min([d_sheep, d_n_forest], key= abs)*s_to_nf*n_forest_pp[round]
        if d_cattle < 0:
            m_change += min([d_cattle, d_sheep], key= abs)*c_to_s*sheep_pp[round]
            m_change += min([d_cattle, d_n_forest], key= abs)*c_to_nf *n_forest_pp[round]
        # money from the area
        m_area = (area_sheep[round] * sheep_pp[round]) + (area_cattle[round] * cattle_pp[round])
        if teamwork == True and teams > round:
            m_teamwork = area_c_forest[round]*c_forest_pp[round] + area_n_forest[round]*n_forest_pp[round]
        if brexit > round:
            m_brexit = (subsidies-1)*(area_sheep[round] * sheep_pp[round])
            if d_n_forest > 0:
                m_brexit += d_n_forest * (subsidies-1)
        m_tourism = tourism * m_area
        # maybe return later on the performance of each landuse/industrie --> append() so that its easy to plot?
        money = m_area + m_change + m_tourism + m_teamwork + brexit
    
    return money

In [85]:


def money_pp_forester(round, tourism, teams, brexit, teamwork, area_sheep, area_c_forest, area_n_forest, sheep_pp, n_forest_pp, c_forest_pp, nf_to_cf, cf_to_nf, subsidies, starting_capital):

# '''
#     area_c_forest: number of pixel displaying commercial forest
#     area_n_forest: number of pixel displaying native forest
#     round: round of the game (starting at 0)
# '''
    if round == 0: 
        money = starting_capital
    else:   
        d_n_forest = area_n_forest[round] - area_n_forest[round-1] # necessary to potentially allow two changes (i.e. a rise or native forests and cattle on cost of sheep )
        m_change = 0
        m_brexit = 0
        # ich habe momentan gemacht, dass man nur etwas verkleinern darf!
        if d_n_forest < 0: 
            m_change += d_c_forest * nf_to_cf * c_forest_pp[round]
        if d_n_forest > 0:
            m_change += d_n_forest * cf_to_nf *n_forest_pp[round]
        # money from the area
        m_area = (area_n_forest[round] * n_forest_pp[round]) + ((area_c_forest[round] * c_forest_pp[round]))
        if teamwork == True and teams > round:
            m_teamwork = area_sheep[round]*sheep_pp[round]
        if brexit > round:
            if d_n_forest > 0:
                m_brexit = d_n_forest * (subsidies-1) + (subsidies-1)*(area_sheep[round] * sheep_pp[round])
        m_tourism = tourism * m_area
        # maybe return later on the performance of each landuse/industrie --> append() so that its easy to plot?
        money = m_area*t + m_change + m_tourism + m_teamwork + brexit
    
    return money

In [ ]:
a,b,c,d = main(rounds, teams, brexit, lulc_matrix, tourism_matrix, n_blocks, rows)

[]
4


IndexError: list index out of range